### Images processing
In this section we are loading our dataset for predictions, in this case we hace 3 datasets:

* Train
* Test
* Validation

So far, we are using just the train dataset. The first step is define the meaning of outfit, as we have describe previously, an oufit wil be define as:

**Men**

* Tshirt and Polos
* Pullover (optional)
* Shorts or pants or jeans 
* Shoes or sneakers 

**Women**
* Top
* Bottom
* Pullover (optional)
* Outerwear (optional)
* Shoes

### Model training
* Use the outfit descriptions to extract only the images that match articles of clothing in our definition of an "outfit."
* Train an autoencoder model for each type of clothing in our defined “outfit.”
* Predict embeddings for each input image.
* Train a multi-input CNN to predict the likes for each outfit.

In [1]:
import pandas as pd
import numpy as np

import json

In [2]:
# Loading the dataset
train_path = '../Datasets/train_no_dup.json'
with open(train_path) as file:
  df_train = json.load(file)


# Showing
display(df_train[0])

{'name': 'Casual',
 'views': 8743,
 'items': [{'index': 1,
   'name': 'mock neck embroidery suede sweatshirt',
   'price': 24.0,
   'likes': 10,
   'image': 'http://img2.polyvoreimg.com/cgi/img-thing?.out=jpg&size=m&tid=194508109',
   'categoryid': 4495},
  {'index': 2,
   'name': 'luxe double zip hooded jacket',
   'price': 150.0,
   'likes': 2250,
   'image': 'http://img2.polyvoreimg.com/cgi/img-thing?.out=jpg&size=m&tid=188778349',
   'categoryid': 25},
  {'index': 3,
   'name': 'citizens humanity high rise rocket hem jean',
   'price': 248.0,
   'likes': 2437,
   'image': 'http://img1.polyvoreimg.com/cgi/img-thing?.out=jpg&size=m&tid=188977857',
   'categoryid': 27},
  {'index': 4,
   'name': 'suede tie short boots',
   'price': 37.0,
   'likes': 2,
   'image': 'http://img1.polyvoreimg.com/cgi/img-thing?.out=jpg&size=m&tid=194942557',
   'categoryid': 261},
  {'index': 5,
   'name': 'cloth travel school backpack',
   'price': 22.0,
   'likes': 2,
   'image': 'http://img2.polyvoreim

## Gender

It is neccesary to keep in mind the importance of divide the genders during the training, that is because the clothes are different so try to get an outfit combined the clothes designed for a male with cloths for a female, could be get bad recommendation due to the pattern. That's why is mandatory filter the womens clothes and then just keep the outfit that matches with out definition of outfit.

### Females
The keys to filter are:
* A menswear look
* Man's
* man's
* men's fashion

### Males
They keys to filter are:
* women' s Fashion
* women's clothing 
* women 
* female
* woman


The principal problem about this is the lack description regarding the gender. That's why the filter by gender is just reduced a little bit for each gender.


In [3]:
for i in range(10):
    display(df_train[i]['desc'])

'A fashion look from January 2017 by beebeely-look featuring Fuji, Citizens of Humanity, casual, casualoutfit, Packandgo, winterjacket and gearbest'

'A fashion look from April 2014 featuring destroyed shirt, skinny fit jeans and vans shoes. Browse and shop related looks.'

'12.19.14'

'A fashion look from January 2016 by alyssaclair-winchester featuring Forever 21, Yves Saint Laurent, Allurez and Plantronics'

'A fashion look from August 2016 by kate-goida featuring Alice + Olivia, Etro, Casetify, Lime Crime and Oscar de la Renta'

'A fashion look from June 2016 by sara-86 featuring New Look, MANGO, Sophia Webster, MICHAEL Michael Kors and Origins'

'A fashion look from February 2017 by exco featuring Isabel Marant, clean, organized, yoins, yoinscollection and loveyoins'

'A fashion look from February 2017 by rasa-j featuring Oasis, Michelle Mason, ChloÃ©, Giuseppe Zanotti, Deborah Lippmann, Natural Life and womensFashion'

'A fashion look from December 2015 by mcheffer featuring Valentino, RED Valentino, valentino, fashiontrend and runwaylook'

'A fashion look from December 2016 by beebeely-look featuring Topshop, Forever 21, Bobbi Brown Cosmetics, Zoku, casual, stripes, fauxfur, fauxfurcoat and yoinsco...'

In [4]:
# Filtering men's clothles, only keep mens fashion
substrings_filter_men = ["men's", "Men", "man", 'Man', "men's fashion", "menswear look"]
women_fashion = filter(lambda x: not any(substring in x['desc'] for substring in substrings_filter_men), df_train)
women_fashion = list(women_fashion)


# Filtering women's clothles, only keep mens fashion
substrings_filter_woman = ["women", "woman", "women's", 'Women', "Woman", "Girl", "girl's", "Girl's", "women's fashion"]
men_fashion = filter(lambda x: not any(substring in x['desc'] for substring in substrings_filter_woman), df_train)
men_fashion = list(men_fashion)


# Amount:
# Number of women's outfits:
print(len(women_fashion))

# Number of men's oufits
print(len(men_fashion))

# The total of outfits
print(len(df_train))

15126
16824
17316


### Filtering by outfit definition

The next step is just keep the outfits that matches with our outfit definition, additional this help to tune the filter by gender.

Whe have the following categories that correspond for women clothing:
* 3 Dresses
* 4 Day Dresses
* 5 Cocktail Dresses
* 7 Skirts
* 8 Mini Skirts
* 9 Knee Length Skirts
* 10 Long Skirts
* 11 Tops
* 15 Tunics
* 17 Blouses
* 18 Cardigans
* 19 Sweaters
* 21 T-Shirts
* 23 Outerwear
* 24 Coats
* 25 Jackets
* 27 Jeans
* 26 Vests
* 28 Pants
* 29 Shorts
* 31 Swimwear
* 41 Shoes
* 49 Sneakers
* 52 Belts
* 101 Maternity
* 104 Tank Tops
* 108 Intimates
* 127 Dressers
* 236 Blazers
* 237 Skinny Jeans
* 238 Bootcut Jeans
* 239 Wide Leg Jeans
* 241 Leggings
* 243 Jumpsuits
* 244 Rompers
* 245 Bras
* 246 Panties & Thongs
* 247 Camisoles
* 252 Activewear Tops
* 253 Activewear Pants
* 254 Activewear Skirts
* 255 Activewear Shorts
* 256 Activewear Jackets
* 257 Sports Bras
* 258 Messenger Bags
* 261 Ankle Booties
* 262 Mid Calf Boots
* 263 Knee High Boots
* 264 Over-The-Knee Boots
* 268 Athletic Shoes
* 309 Activewear Tank Tops
* 306 Suspenders
* 4516 Wedding Dresses
* 4517 Bikini Tops
* 4518 Bikini Bottoms
* 1605 Bikinis
* 4495 Sweatshirts
* 4496 Hoodies

We can classify that categories and reduce in general categories:

**Women**
* Dress
    * 3 Dresses
    * 4 Day Dresses
    * 5 Cocktail Dresses

* Top
    * 11 Tops
    * 15 Tunics
    * 17 Blouses
    * 21 T-Shirts
    * 104 Tank Tops
    * 244 Rompers
    * 245 Bras
    * 247 Camisoles
    * 252 Activewear Tops
    * 257 Sports Bras
    * 309 Activewear Tank Tops
    * 4517 Bikini Tops

* Bottom
    * 7 Skirts
    * 8 Mini Skirts
    * 9 Knee Length Skirts
    * 10 Long Skirts
    * 27 Jeans
    * 28 Pants
    * 29 Shorts
    * 237 Skinny Jeans
    * 238 Bootcut Jeans
    * 239 Wide Leg Jeans
    * 241 Leggings
    * 253 Activewear Pants
    * 254 Activewear Skirts
    * 255 Activewear Shorts
    * 4518 Bikini Bottoms

* Pullover (optional)
    * 19 Sweaters
    * 4495 Sweatshirts
    * 4496 Hoodies

* Outerwear (optional)
    * 18 Cardigans
    * 23 Outerwear
    * 24 Coats
    * 25 Jackets
    * 26 Vests
    * 236 Blazers
    * 256 Activewear Jackets

* Shoes
    * 41 Shoes
    * 49 Sneakers
    * 261 Ankle Booties
    * 262 Mid Calf Boots
    * 263 Knee High Boots
    * 264 Over-The-Knee Boots
    * 268 Athletic Shoes

* Belts (optional)
    * 52 Belts


In [18]:
# Filtering outfits for women fashion
# Categories
numbers_list = [
    2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 17, 18, 19, 21, 23, 24, 25, 26, 27, 28, 29, 
    41, 49, 52, 104, 236, 237, 238, 239, 241, 
    244, 245, 247, 252, 253, 254, 255, 256, 257, 261, 262, 263, 264, 309, 
    4517, 4518, 4241, 240, 241, 250, 251, 4495, 4496
]

category_numbers = [3, 23, 41, 7, 11, 27, 25]

# Crear una lista para almacenar los outfits que cumplen con el criterio
womens_fashion = []

# Iterar sobre cada outfit en women_fashion
for outfit in women_fashion:
    # Obtener los 'categoryid' de todos los items del outfit
    outfit_categories = {item['categoryid'] for item in outfit['items']}
    # Verificar si todos los 'categoryid' del outfit están en category_numbers
    if outfit_categories.issubset(numbers_list) and len(outfit_categories)> 3:
        # Si todos los 'categoryid' del outfit están en category_numbers, agregar el outfit a la lista filtrada
        womens_fashion.append(outfit)

# Mostrar la longitud de la lista filtrada
print(len(womens_fashion))


113


### Getting the images form each outfit.

In [6]:
import os
import cv2

def buscar_directorio(nombre_directorio, ruta_carpeta):
    # Recorre todos los elementos (archivos y directorios) en la ruta especificada
    for root, dirs, files in os.walk(ruta_carpeta):
        # Verifica si el nombre del directorio está en la lista de directorios actuales
        if nombre_directorio in dirs:
            # Combina el directorio raíz con el nombre del directorio para obtener la ruta completa
            ruta_completa = os.path.join(root, nombre_directorio)
            # Imprime la ruta completa como una cadena de texto
            print("Ruta completa:", ruta_completa)
            # Si solo necesitas la ruta como una cadena de texto, puedes retornarla en lugar de imprimirla
            return ruta_completa

# Especifica el nombre del directorio que deseas buscar
nombre_directorio = "209060708"
# Especifica la ruta de la carpeta en la que deseas buscar
ruta_carpeta = "../Datasets/images"

# Llama a la función para buscar el directorio
ruta = buscar_directorio(nombre_directorio, ruta_carpeta)
ruta = str(ruta).replace("\\","/")+'/2.jpg'
print(ruta)

# Cargar la imagen
imagen = cv2.imread(ruta)

# Mostrar la imagen en una ventana
cv2.imshow('Imagen', imagen)
cv2.waitKey(2000)
cv2.destroyAllWindows()


Ruta completa: ../Datasets/images\209060708
../Datasets/images/209060708/2.jpg


In [7]:
import os

def buscar_directorio(nombre_directorio, ruta_carpeta):
    for root, dirs, files in os.walk(ruta_carpeta):
        if nombre_directorio in dirs:
            ruta_completa = os.path.join(root, nombre_directorio)
            return ruta_completa

route = "../Datasets/images"
for i in range(len(womens_fashion)):
    id_outfit = womens_fashion[i]['set_id']
    dir_searched = str(id_outfit)

    route_relative = buscar_directorio(dir_searched, route)
    

    for j in range(len(womens_fashion[i]['items'])):
        number = womens_fashion[i]['items'][j]['index']
        route_complete = str(route_relative).replace("\\","/")+'/'+str(number)+'.jpg'
        # print(route_complete)

        # Substitution
        womens_fashion[i]['items'][j]['image'] = route_complete

In [8]:
womens_fashion

[{'name': 'Untitled #34',
  'views': 206,
  'items': [{'index': 1,
    'name': 'miller striped shirt boyfriend tee',
    'price': 92.0,
    'likes': 12792,
    'image': '../Datasets/images/122754559/1.jpg',
    'categoryid': 21},
   {'index': 2,
    'name': "doublju women's faux leather power shoulder jacket",
    'price': 40.0,
    'likes': 15454,
    'image': '../Datasets/images/122754559/2.jpg',
    'categoryid': 25},
   {'index': 3,
    'name': 'shredded black high waist jeans edgy clothing',
    'price': 54.0,
    'likes': 5602,
    'image': '../Datasets/images/122754559/3.jpg',
    'categoryid': 237},
   {'index': 4,
    'name': 'converse sneakers classic canvas black',
    'price': 53.0,
    'likes': 12531,
    'image': '../Datasets/images/122754559/4.jpg',
    'categoryid': 49}],
  'image': 'http://ak1.polyvoreimg.com/cgi/img-set/cid/122754559/id/VAZyQPcG5BG2Z5nHzSGTQA/size/y.jpg',
  'likes': 8,
  'date': 'Two years',
  'set_url': 'http://www.polyvore.com/untitled_34/set?id=122

### Conservando solo los directorios de interés
Agregado espóntaneo

In [20]:
import os
import shutil

def crear_directorios_desde_json(womens_fashion, origen, destino):
    for outfit in womens_fashion:
        id_outfit = outfit['set_id']
        dir_searched = str(id_outfit)
        route_relative = buscar_directorio(dir_searched, origen)

        if os.path.exists(route_relative):
            destino_directorio = os.path.join(destino, dir_searched)
            shutil.copytree(route_relative, destino_directorio)

origen = "../Datasets/images"
destino = "../Datasets/images_filtered"
crear_directorios_desde_json(womens_fashion, origen, destino)


### Showing the first outfit


In [19]:
for j in range(len(womens_fashion[0]['items'])):   
    print(womens_fashion[0]['items'][j]['image'])
    imagen = cv2.imread(womens_fashion[0]['items'][j]['image'])

    # Mostrar la imagen en una ventana
    cv2.imshow('Imagen', imagen)
    cv2.waitKey(2000)
    cv2.destroyAllWindows()

http://img1.polyvoreimg.com/cgi/img-thing?.out=jpg&size=m&tid=148184208


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


### Saving the Json

In [10]:
output_file = "../Datasets/womens_fashion_defined.json"

# Guardar el archivo filtrado en formato JSON
with open(output_file, "w") as outfile:
    json.dump(womens_fashion, outfile)

print("Json Saved!:", output_file)

Json Saved!: ../Datasets/womens_fashion.json
